In [6]:
from pathlib import Path

DATA_PATH = Path(r"C:\Users\areesa\Documents\Urdu_GLUE_xlm_roberta\data\raw\uner.txt")

print("File exists:", DATA_PATH.exists())

with DATA_PATH.open("r", encoding="utf-16") as f:
    for i in range(40):
        line = f.readline()
        if not line:
            break
        print(line.rstrip())


File exists: True
<LOCATION>برطانیہ</LOCATION> کے سابق فوجی سربراہ کا کہنا ہے کہ فوجی حکام نے <LOCATION>افغانستان</LOCATION> میں دہشت گردی کے خلاف جنگ کی شدت کا غلط اندازہ لگایا تھا۔
<ORGANIZATION>بی بی سی</ORGANIZATION> کو دیے گئے ایک خصوصی انٹرویو میں <DESIGNATION>جنرل</DESIGNATION> <PERSON>سر پیٹر وال</PERSON> کا کہنا تھا کہ ان کے خیال میں <LOCATION>افغانستان</LOCATION> میں ’محدود‘ مقاصد حاصل کرنے کے لیے موجود ’فوجی افرادی قوت کافی‘ تھی، لیکن اب وہ سوچتے ہیں کہ انھوں نے غلط اندازہ لگایا تھا۔
<DESIGNATION>بریگیڈیئر</DESIGNATION> <PERSON>ایڈ بٹلر</PERSON> <DATE>سنہ دوہزارچھ</DATE> میں <LOCATION>ہلمند</LOCATION> کے فوجی کمانڈر تھے۔
ان کے بقول نہ تو ان کے فوجی اس مشکل جنگ کے لیے تیار تھے اور نہ ان کے پاس لڑائی کے لیے درکار اسلحہ موجود تھا۔
لیکن برطانوی وزارت دفاع کا کہنا ہے کہ انھیں <LOCATION>افغانستان</LOCATION> میں حاصل کی گئی فتوحات پر فخر ہے۔
میرے پیش کردہ منصوبے میں <LOCATION>افغانستان</LOCATION> بھیجے گئے فوجیوں کی تعداد کو محدود مقاصد کے حصول کے لیے کافی قرار دیا گیا تھا لیکن ابھ

In [7]:
# 2.1 Read full text safely
from pathlib import Path

DATA_PATH = Path(r"C:\Users\areesa\Documents\Urdu_GLUE_xlm_roberta\data\raw\uner.txt")

with DATA_PATH.open("r", encoding="utf-16") as f:
    raw_text = f.read()

print(raw_text[:1000])


<LOCATION>برطانیہ</LOCATION> کے سابق فوجی سربراہ کا کہنا ہے کہ فوجی حکام نے <LOCATION>افغانستان</LOCATION> میں دہشت گردی کے خلاف جنگ کی شدت کا غلط اندازہ لگایا تھا۔
<ORGANIZATION>بی بی سی</ORGANIZATION> کو دیے گئے ایک خصوصی انٹرویو میں <DESIGNATION>جنرل</DESIGNATION> <PERSON>سر پیٹر وال</PERSON> کا کہنا تھا کہ ان کے خیال میں <LOCATION>افغانستان</LOCATION> میں ’محدود‘ مقاصد حاصل کرنے کے لیے موجود ’فوجی افرادی قوت کافی‘ تھی، لیکن اب وہ سوچتے ہیں کہ انھوں نے غلط اندازہ لگایا تھا۔
<DESIGNATION>بریگیڈیئر</DESIGNATION> <PERSON>ایڈ بٹلر</PERSON> <DATE>سنہ دوہزارچھ</DATE> میں <LOCATION>ہلمند</LOCATION> کے فوجی کمانڈر تھے۔
ان کے بقول نہ تو ان کے فوجی اس مشکل جنگ کے لیے تیار تھے اور نہ ان کے پاس لڑائی کے لیے درکار اسلحہ موجود تھا۔
لیکن برطانوی وزارت دفاع کا کہنا ہے کہ انھیں <LOCATION>افغانستان</LOCATION> میں حاصل کی گئی فتوحات پر فخر ہے۔
میرے پیش کردہ منصوبے میں <LOCATION>افغانستان</LOCATION> بھیجے گئے فوجیوں کی تعداد کو محدود مقاصد کے حصول کے لیے کافی قرار دیا گیا تھا لیکن ابھی میں کھل کر اپنے 

In [8]:
# 2.2 Extract tagged and non-tagged text
import re

TAG_PATTERN = re.compile(r"<(.*?)>(.*?)</\1>", re.DOTALL)

def xml_to_bio(text):
    tokens = []
    labels = []

    pos = 0
    for match in TAG_PATTERN.finditer(text):
        # Handle text before tag
        before = text[pos:match.start()]
        for tok in before.split():
            tokens.append(tok)
            labels.append("O")

        tag = match.group(1)
        content = match.group(2)

        content_tokens = content.split()
        for i, tok in enumerate(content_tokens):
            prefix = "B-" if i == 0 else "I-"
            tokens.append(tok)
            labels.append(prefix + tag)

        pos = match.end()

    # Remaining text
    after = text[pos:]
    for tok in after.split():
        tokens.append(tok)
        labels.append("O")

    return tokens, labels


In [9]:
# 2.3 Test conversion on a small sample
sample_text = "<PERSON>سر پیٹر وال</PERSON> نے <LOCATION>افغانستان</LOCATION> کا دورہ کیا"

toks, labs = xml_to_bio(sample_text)

for t, l in zip(toks, labs):
    print(f"{t}\t{l}")


سر	B-PERSON
پیٹر	I-PERSON
وال	I-PERSON
نے	O
افغانستان	B-LOCATION
کا	O
دورہ	O
کیا	O


In [10]:
# 3.1 Sentence splitting (Urdu-safe)
import re

def split_urdu_sentences(text):
    # split on Urdu full stop
    sentences = re.split(r"۔\s*", text)
    return [s.strip() for s in sentences if s.strip()]


In [11]:
# 3.2 Convert full dataset to BIO sentences
all_sentences = split_urdu_sentences(raw_text)

bio_sentences = []

for sent in all_sentences:
    tokens, labels = xml_to_bio(sent)
    if len(tokens) > 0:
        bio_sentences.append({
            "tokens": tokens,
            "ner_tags": labels
        })

print("Total sentences:", len(bio_sentences))
print("Sample sentence:")
print(bio_sentences[0])


Total sentences: 1738
Sample sentence:
{'tokens': ['برطانیہ', 'کے', 'سابق', 'فوجی', 'سربراہ', 'کا', 'کہنا', 'ہے', 'کہ', 'فوجی', 'حکام', 'نے', 'افغانستان', 'میں', 'دہشت', 'گردی', 'کے', 'خلاف', 'جنگ', 'کی', 'شدت', 'کا', 'غلط', 'اندازہ', 'لگایا', 'تھا'], 'ner_tags': ['B-LOCATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOCATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}


In [12]:
# 3.3 Quick sanity check
for i in range(3):
    print("\nSentence", i)
    for t, l in zip(bio_sentences[i]["tokens"], bio_sentences[i]["ner_tags"]):
        print(f"{t:15s} {l}")




Sentence 0
برطانیہ         B-LOCATION
کے              O
سابق            O
فوجی            O
سربراہ          O
کا              O
کہنا            O
ہے              O
کہ              O
فوجی            O
حکام            O
نے              O
افغانستان       B-LOCATION
میں             O
دہشت            O
گردی            O
کے              O
خلاف            O
جنگ             O
کی              O
شدت             O
کا              O
غلط             O
اندازہ          O
لگایا           O
تھا             O

Sentence 1
بی              B-ORGANIZATION
بی              I-ORGANIZATION
سی              I-ORGANIZATION
کو              O
دیے             O
گئے             O
ایک             O
خصوصی           O
انٹرویو         O
میں             O
جنرل            B-DESIGNATION
سر              B-PERSON
پیٹر            I-PERSON
وال             I-PERSON
کا              O
کہنا            O
تھا             O
کہ              O
ان              O
کے              O
خیال            O
میں             O
افغانستان       B-LOCA

In [14]:
# 4.1 Build label ↔ id mappings 

unique_labels = set()

for sent in bio_sentences:
    unique_labels.update(sent["ner_tags"])

label_list = sorted(unique_labels)

label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

print("Label to ID mapping:")
for k, v in label2id.items():
    print(f"{k:15s} -> {v}")


Label to ID mapping:
B-DATE          -> 0
B-DESIGNATION   -> 1
B-LOCATION      -> 2
B-NUMBER        -> 3
B-ORGANIZATION  -> 4
B-PERSON        -> 5
B-TIME          -> 6
I-DATE          -> 7
I-DESIGNATION   -> 8
I-LOCATION      -> 9
I-NUMBER        -> 10
I-ORGANIZATION  -> 11
I-PERSON        -> 12
I-TIME          -> 13
O               -> 14


In [15]:
# 4.2 Convert labels to IDs
for sent in bio_sentences:
    sent["ner_tag_ids"] = [label2id[l] for l in sent["ner_tags"]]


In [16]:
#4.3 Create HuggingFace Dataset
from datasets import Dataset

hf_dataset = Dataset.from_list([
    {
        "tokens": sent["tokens"],
        "ner_tags": sent["ner_tag_ids"]
    }
    for sent in bio_sentences
])

hf_dataset


C:\Users\areesa\anaconda3\envs\urdu_glue_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 1738
})

In [17]:
# 4.4 Final dataset check
hf_dataset[0]


{'tokens': ['برطانیہ',
  'کے',
  'سابق',
  'فوجی',
  'سربراہ',
  'کا',
  'کہنا',
  'ہے',
  'کہ',
  'فوجی',
  'حکام',
  'نے',
  'افغانستان',
  'میں',
  'دہشت',
  'گردی',
  'کے',
  'خلاف',
  'جنگ',
  'کی',
  'شدت',
  'کا',
  'غلط',
  'اندازہ',
  'لگایا',
  'تھا'],
 'ner_tags': [2,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  2,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14]}

In [18]:
# 5.1 Load tokenizer (XLM-R Large)
from transformers import AutoTokenizer

MODEL_NAME = "xlm-roberta-large"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


In [19]:
# 5.2 Tokenize + align label
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding=False,
        max_length=128,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)   # ignored in loss
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx])  # same label for subwords

            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [20]:
# 5.3 Apply tokenization to dataset
tokenized_dataset = hf_dataset.map(
    tokenize_and_align_labels,
    batched=True
)


Map: 100%|████████████████████████████████████████████████████████████████████████████| 1738/1738 [00:00<00:00, 15871.19 examples/s]


In [21]:
# 5.4 Final tokenization sanity check
example = tokenized_dataset[0]

print("Tokens:")
print(tokenizer.convert_ids_to_tokens(example["input_ids"]))

print("\nLabels:")
print(example["labels"])


Tokens:
['<s>', '▁برطانیہ', '▁کے', '▁سابق', '▁فوجی', '▁سربراہ', '▁کا', '▁کہنا', '▁ہے', '▁کہ', '▁فوجی', '▁حکام', '▁نے', '▁افغانستان', '▁میں', '▁دہشت', '▁گردی', '▁کے', '▁خلاف', '▁جنگ', '▁کی', '▁شدت', '▁کا', '▁غلط', '▁اندازہ', '▁لگایا', '▁تھا', '</s>']

Labels:
[-100, 2, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 2, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, -100]


In [22]:
# 6.1 Create train / eval split for Zero-Shot
from datasets import DatasetDict

split_dataset = tokenized_dataset.train_test_split(
    test_size=0.2,
    seed=42
)

split_dataset


DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1390
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 348
    })
})

In [23]:
# 6.2 Load XLM-RoBERTa for Token Classification
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    "xlm-roberta-large",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
# 6.3 FREEZE the encoder
for param in model.roberta.parameters():
    param.requires_grad = False


In [32]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding=True
)


In [26]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./ner_zero_shot",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=20,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=True,
    seed=42
)


In [28]:
!pip install seqeval


  Using cached seqeval-1.2.2-py3-none-any.whl


In [29]:

import numpy as np
from seqeval.metrics import f1_score
from sklearn.metrics import accuracy_score

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = []
    true_labels = []

    for pred, lab in zip(predictions, labels):
        filtered_pred = []
        filtered_lab = []
        for p_i, l_i in zip(pred, lab):
            if l_i != -100:
                filtered_pred.append(id2label[p_i])
                filtered_lab.append(id2label[l_i])
        true_predictions.append(filtered_pred)
        true_labels.append(filtered_lab)

    # Entity-level F1 (NER standard)
    f1 = f1_score(true_labels, true_predictions)

    # Token-level Accuracy
    accuracy = accuracy_score(
        [l for sent in true_labels for l in sent],
        [p for sent in true_predictions for p in sent]
    )

    return {
        "f1": f1,
        "accuracy": accuracy
    }


In [34]:
# 6.6 Trainer (Zero-Shot)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,  
    compute_metrics=compute_metrics
)


C:\Users\areesa\AppData\Local\Temp\ipykernel_34080\851354905.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [35]:
trainer.train()


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,No log,3.146254,0.007302,0.022994
2,No log,3.033601,0.007299,0.022994
3,3.155800,2.875426,0.007335,0.022994
4,3.155800,2.727278,0.007335,0.022994
5,2.857900,2.591779,0.008918,0.023165
6,2.857900,2.467337,0.009728,0.023420
7,2.558900,2.353962,0.009642,0.023846
8,2.558900,2.251660,0.010004,0.034236
9,2.558900,2.160440,0.009424,0.076307
10,2.318600,2.079285,0.016183,0.158491


TrainOutput(global_step=880, training_loss=2.29835336858576, metrics={'train_runtime': 185.7474, 'train_samples_per_second': 149.666, 'train_steps_per_second': 4.738, 'total_flos': 3935928502899612.0, 'train_loss': 2.29835336858576, 'epoch': 20.0})

In [36]:
# Final evaluation on the validation set
zero_shot_results = trainer.evaluate()

print("Zero-Shot NER Results (XLM-R Large):")
print(f"F1 Score     : {zero_shot_results['eval_f1']:.4f}")
print(f"Accuracy     : {zero_shot_results['eval_accuracy']:.4f}")


Zero-Shot NER Results (XLM-R Large):
F1 Score     : 0.0257
Accuracy     : 0.5340


In [37]:
# 7.1 Identify sentences containing each entity class
from collections import defaultdict

# entity classes (ignore O)
entity_classes = sorted(
    {label.replace("B-", "").replace("I-", "")
     for label in label_list if label != "O"}
)

print("Entity classes:", entity_classes)

# Map entity -> sentence indices
entity_to_sentences = defaultdict(list)

for idx, sent in enumerate(bio_sentences):
    labels = sent["ner_tags"]
    for ent in entity_classes:
        if f"B-{ent}" in labels:
            entity_to_sentences[ent].append(idx)


Entity classes: ['DATE', 'DESIGNATION', 'LOCATION', 'NUMBER', 'ORGANIZATION', 'PERSON', 'TIME']


In [38]:
# 7.2 Take first 16 usable sentences per class
SHOT = 16

selected_indices = set()

for ent in entity_classes:
    usable = entity_to_sentences[ent][:SHOT]
    print(f"{ent}: {len(usable)} sentences")
    selected_indices.update(usable)

selected_indices = sorted(selected_indices)
print("\nTotal selected sentences:", len(selected_indices))


DATE: 16 sentences
DESIGNATION: 16 sentences
LOCATION: 16 sentences
NUMBER: 16 sentences
ORGANIZATION: 16 sentences
PERSON: 16 sentences
TIME: 16 sentences

Total selected sentences: 75


In [39]:
# 7.3 Build the 16-shot dataset
from datasets import Dataset

shot16_dataset = Dataset.from_list([
    tokenized_dataset[i] for i in selected_indices
])

shot16_dataset


Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 75
})

In [40]:
# 7.4 Create a small validation split (for early stopping)
shot16_split = shot16_dataset.train_test_split(
    test_size=0.2,
    seed=42
)

shot16_split


DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 60
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 15
    })
})

In [41]:
# STEP 7.5 — Load Model (UNFROZEN)
from transformers import AutoModelForTokenClassification

model_16shot = AutoModelForTokenClassification.from_pretrained(
    "xlm-roberta-large",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
# STEP 7.6 — Training Arguments
from transformers import TrainingArguments

training_args_16 = TrainingArguments(
    output_dir="./ner_16shot",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=20,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=True,
    seed=42
)


In [43]:
# STEP 7.7 — Trainer
from transformers import Trainer

trainer_16shot = Trainer(
    model=model_16shot,
    args=training_args_16,
    train_dataset=shot16_split["train"],
    eval_dataset=shot16_split["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


C:\Users\areesa\AppData\Local\Temp\ipykernel_34080\533048788.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_16shot = Trainer(


In [44]:
# 7.8 — Train 16-Shot NER
trainer_16shot.train()


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,No log,2.874883,0.018265,0.026515
2,No log,2.874883,0.018265,0.026515
3,No log,2.874883,0.018265,0.026515
4,No log,2.656768,0.023438,0.032197
5,No log,1.981714,0.000000,0.757576
6,No log,1.269806,0.000000,0.782197
7,No log,1.243556,0.000000,0.782197
8,No log,1.068496,0.000000,0.782197
9,No log,0.914111,0.000000,0.782197
10,No log,0.867547,0.000000,0.782197


TrainOutput(global_step=40, training_loss=1.3729832649230957, metrics={'train_runtime': 1048.0889, 'train_samples_per_second': 1.145, 'train_steps_per_second': 0.038, 'total_flos': 165977412651000.0, 'train_loss': 1.3729832649230957, 'epoch': 20.0})

In [45]:
# 7.9 — Extract Final 16-Shot Results
results_16shot = trainer_16shot.evaluate()

print("16-Shot NER Results (XLM-R Large):")
print(f"F1 Score     : {results_16shot['eval_f1']:.4f}")
print(f"Accuracy     : {results_16shot['eval_accuracy']:.4f}")


16-Shot NER Results (XLM-R Large):
F1 Score     : 0.2105
Accuracy     : 0.8277


In [46]:
# 80/20 FULL NER FINE-TUNING

In [47]:
# 8.1 Create 80/20 split (FULL DATA)
full_split = tokenized_dataset.train_test_split(
    test_size=0.2,
    seed=42
)

full_split


DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1390
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 348
    })
})

In [48]:
#8.2 Load fresh model (UNFROZEN)
from transformers import AutoModelForTokenClassification

model_80 = AutoModelForTokenClassification.from_pretrained(
    "xlm-roberta-large",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
# 8.3 Training Arguments 
from transformers import TrainingArguments

training_args_80 = TrainingArguments(
    output_dir="./ner_80_20",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=20,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=True,
    seed=42
)


In [51]:
# 8.4 Trainer (80/20)
from transformers import Trainer

trainer_80 = Trainer(
    model=model_80,
    args=training_args_80,
    train_dataset=full_split["train"],
    eval_dataset=full_split["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


C:\Users\areesa\AppData\Local\Temp\ipykernel_34080\1676567414.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_80 = Trainer(


In [52]:
trainer_80.train()


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,No log,0.720599,0.000000,0.807954
2,No log,0.216297,0.638153,0.935360
3,1.121300,0.142670,0.815910,0.960484
4,1.121300,0.109028,0.851408,0.966275
5,0.121500,0.107293,0.860814,0.968149
6,0.121500,0.108408,0.864846,0.968660
7,0.062300,0.128720,0.866856,0.967638
8,0.062300,0.106707,0.881237,0.971725
9,0.062300,0.123203,0.877180,0.969426
10,0.037100,0.133004,0.873530,0.970107


TrainOutput(global_step=880, training_loss=0.15820768414573236, metrics={'train_runtime': 2593.159, 'train_samples_per_second': 10.721, 'train_steps_per_second': 0.339, 'total_flos': 3935928502899612.0, 'train_loss': 0.15820768414573236, 'epoch': 20.0})

In [53]:
results_80 = trainer_80.evaluate()

print("80/20 NER Results (XLM-R Large):")
print(f"F1 Score     : {results_80['eval_f1']:.4f}")
print(f"Accuracy     : {results_80['eval_accuracy']:.4f}")


80/20 NER Results (XLM-R Large):
F1 Score     : 0.8849
Accuracy     : 0.9718
